# Task 2: Create Bengali word embeddings

## Import libraries

In [1]:
# Imports
import re
import string
import json
from datetime import datetime
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import Module
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords

device = 'cuda'

import random

torch.manual_seed(123)
torch.cuda.manual_seed(234)
np.random.seed(345)
random.seed(456)
torch.manual_seed(567)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

## Load data

In [2]:
ben_train_df = pd.read_csv('../hindi_bengali/save/bengali_hatespeech_sample_train_preprocessed.csv')
ben_test_df = pd.read_csv('../hindi_bengali/save/bengali_hatespeech_sample_test_preprocessed.csv')

display(ben_train_df.head())

,sentence,hate,category
0,সমকামী হুজুর,0,religion
1,ছাএলীগ সালা দের নিসিদ্দ হক,1,politics
2,কাওয়া গদি ছারলে বুজবে জুতা কেমনে খায়,0,politics
3,কাউয়া কাদের বড় মাগীখোর ভিডিও পিক দেখলে বুঝা লু...,1,politics
4,অপু ভালো কথা ছোট করবেনা,0,"Meme, TikTok and others"


In [3]:
# train data:
# remove empty texts
ben_train_df = ben_train_df[ben_train_df.sentence.str.len() > 0]
# extract sentences and labels
train_sentences = [text.split() for text in ben_train_df['sentence']]
train_labels = ben_train_df['hate'].to_numpy()

# test data:
# remove empty texts
ben_test_df = ben_test_df[ben_test_df.sentence.str.len() > 0]
# extract sentences and labels
test_sentences = [text.split() for text in ben_test_df['sentence']]
test_labels = ben_test_df['hate'].to_numpy()

### Print out data/statistics

In [4]:
print('Train data:')
print(train_sentences[:3])
print(train_labels)
print()
print('Test data:')
print(test_sentences[:3])
print(test_labels)

Train data:
[['সমকামী', 'হুজুর'], ['ছাএলীগ', 'সালা', 'দের', 'নিসিদ্দ', 'হক'], ['কাওয়া', 'গদি', 'ছারলে', 'বুজবে', 'জুতা', 'কেমনে', 'খায়']]
[0 1 0 ... 1 0 0]

Test data:
[['মহিলাকে', 'রিমানডে'], ['তুর', 'রিপাতকে', 'মন', 'চাইছিল', 'ছেড়ে', 'গেলি', 'সাথে', 'মিত্যে', 'অভিনয়', 'করলি', 'দুনিয়া', 'উঠালি', 'তুর', 'নরকেও', 'ঠাঁই', 'হবেনা', 'তুই', 'আকাশের', 'মিতুর', 'মত', 'করলি', 'তুর', 'মত', 'বিশ্বাস', 'ঘাতকনীর', 'ফাসি', 'হউক', 'রিফাত', 'তোকে', 'ফেলে', 'যেত', 'হয়তবা', 'বেঁচে', 'যেত', 'সরল', 'ভালবাসাকে', 'হত্যা', 'করলি', 'তুই', 'নারী', 'জাতের', 'কলংক', 'তুকে', 'দেখলে', 'বুঝা', 'আসলে', 'তোর', 'পরিক্ষলপনা', 'তোর', 'চোখ', 'মুখ', 'সাক্ষী', 'তুইয়ি', 'জরিত', 'নারী', 'জাতের', 'কলংক'], ['হুমায়ুন', 'আজাদ', 'এতো', 'বড়', 'ক্রাক', 'মাতাল']]
[0 0 0 ... 0 0 1]


## Prepare vocab set

In [5]:
# load mapping {word -> id} and {id -> word}
with open('../hindi_bengali/save/bengali_word_to_int_dict.json') as f:
    word_to_int = json.load(f)
with open('../hindi_bengali/save/bengali_int_to_word_dict.json') as f:
    int_to_word = json.load(f)
    int_to_word = {int(k) : v for k, v in int_to_word.items()}
with open('../hindi_bengali/save/bengali_word_counter.json') as f:
    word_counter = json.load(f)

# get vocab_size
vocab_size = len(word_to_int)
print(f'vocab_size: {vocab_size}')

# get total occurences
total_words = sum(word_counter.values())
print(f'total word occurences: {total_words}')

vocab_size: 15231
total word occurences: 47897


## Define sub-sampling

In [6]:
def sampling_prob(word):
    z = word_counter[word] / total_words
    p_keep = ((z/0.000001)**0.5 + 1) * (0.000001/z)
    return p_keep

## skip-gram

In [7]:
def get_target_context(sentence: list(str())):
    for i, word in enumerate(sentence):
        for j, context_word in enumerate(sentence[i-window_size:i+window_size+1]):
            if j != i and random.random() < sampling_prob(context_word):
                yield (torch.tensor(word_to_int[word], dtype=torch.long).unsqueeze(0), 
                       torch.tensor(word_to_int[context_word], dtype=torch.long).unsqueeze(0))

# Train word-embeddings

## hyper-parameters

In [8]:
window_size = 10
embedding_size = 300
learning_rate = 0.01
batch_size = 256
epochs = 100

## Model

In [9]:
class Word2Vec(Module):
    def __init__(self):
        super(Word2Vec, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size)
        self.fc = nn.Linear(embedding_size, vocab_size)

    def forward(self, word_id):
        out = self.embed(word_id)
        out = self.fc(out)
        return out.squeeze(1)
    
    def to_embed(self, word_id):
        return self.embed(word_id)
    
word2vec = Word2Vec()
save_path = './save/bengali_word2vec.pt'
torch.save(word2vec.state_dict(), save_path)

display(word2vec.parameters)

<bound method Module.parameters of Word2Vec(
  (embed): Embedding(15231, 300)
  (fc): Linear(in_features=300, out_features=15231, bias=True)
)>

## Loss function and Optimizer

In [10]:
optimizer = optim.Adam(word2vec.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

## Dataset

In [11]:
class W2VDataset(Dataset):
    def __init__(self, sentences):
        self.data = []
        for sentence in sentences:
            for data_point in get_target_context(sentence):
                self.data.append(data_point)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

## Learning parameters

In [12]:
# load initial weights
word2vec.load_state_dict(torch.load(save_path))
word2vec = word2vec.to(device)

# training
early_stop = 5
list_loss = []
for epoch in range(1, epochs+1):
    train_dataset = W2VDataset(train_sentences)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    losses = 0.
    cnt = 0
    word2vec.train()
    for words, context_words in tqdm(train_loader):
        optimizer.zero_grad()
        pred = word2vec(words.to(device))
        loss = criterion(pred, context_words.squeeze(1).to(device))
        loss.backward()
        optimizer.step()
        losses += loss.detach().item() * len(words)
        cnt += len(words)

    epoch_loss = losses / cnt
    print(f'Epoch {epoch:2}: training loss: {epoch_loss:.4f} over {cnt} training points.')
    
    if epoch % 10 == 0:
        # save check-point embedding
        embedding_weights = word2vec.embed.state_dict()
        torch.save(embedding_weights, f'./save/embedding_checkpoints/embedding_weights_{epoch}_epoch_{embedding_size}_dim_{window_size}_wsize.pt')
    
    list_loss.append(epoch_loss)
    if len(list_loss) > early_stop and min(list_loss[-early_stop:]) > min(list_loss[:-early_stop]):
        print('Training loss is not reducing anymore, terminate.')
        break

print("Training finished")

100%|██████████| 189/189 [00:07<00:00, 25.10it/s]


Epoch  1: training loss: 11.4425 over 48288 training points.


100%|██████████| 189/189 [00:07<00:00, 24.88it/s]


Epoch  2: training loss: 11.8694 over 48217 training points.


100%|██████████| 189/189 [00:07<00:00, 26.30it/s]


...

Epoch 98: training loss: 5.7839 over 47957 training points.


100%|██████████| 189/189 [00:07<00:00, 25.72it/s]


Epoch 99: training loss: 5.7883 over 48148 training points.


100%|██████████| 190/190 [00:07<00:00, 24.99it/s]

Epoch 100: training loss: 5.8138 over 48617 training points.
Training finished


In [13]:
# save the word-embedding layer weights
embedding_weights = word2vec.embed.state_dict()
torch.save(embedding_weights, f'save/embedding_weights.pt')